In [33]:
import pandas as pd
from zipline.data import bundles
from zipline.data.data_portal import DataPortal
from zipline.utils.calendars import get_calendar
from zipline.assets._assets import Future
from zipline.utils.run_algo import load_extensions

# Load extensions.py; this allows you access to custom bundles
load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)

# Set-Up Pricing Data Access
trading_calendar = get_calendar('CME')
bundle = 'futures'
bundle_data = bundles.load(bundle)

In [34]:
bundle_data.asset_finder.futures_exchanges

Table('futures_exchanges', MetaData(bind=Engine(sqlite:////Users/jonathan/.zipline/data/futures/2018-09-20T23;09;04.461188/assets-6.sqlite)), Column('exchange', TEXT(), table=<futures_exchanges>, primary_key=True, nullable=False), Column('timezone', TEXT(), table=<futures_exchanges>), schema=None)

In [35]:
bundle_data.asset_finder.futures_root_symbols

Table('futures_root_symbols', MetaData(bind=Engine(sqlite:////Users/jonathan/.zipline/data/futures/2018-09-20T23;09;04.461188/assets-6.sqlite)), Column('root_symbol', TEXT(), table=<futures_root_symbols>, primary_key=True, nullable=False), Column('root_symbol_id', INTEGER(), table=<futures_root_symbols>), Column('sector', TEXT(), table=<futures_root_symbols>), Column('description', TEXT(), table=<futures_root_symbols>), Column('exchange', TEXT(), ForeignKey('futures_exchanges.exchange'), table=<futures_root_symbols>), schema=None)

In [37]:
data = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    future_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader,
)

In [38]:
bundle_data.asset_finder.futures_sids[0:10]

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [39]:
fut = bundle_data.asset_finder.retrieve_futures_contracts([0])[0]

In [40]:
#bundle_data.asset_finder.lookup_future_symbol('ASF18')

In [41]:
fut

Future(0 [A6F18])

In [47]:
end_dt = pd.Timestamp('2018-01-05', tz='UTC', offset='C')
start_dt = pd.Timestamp('2018-01-02', tz='UTC', offset='C')
    
end_loc = trading_calendar.closes.index.get_loc(end_dt)
start_loc = trading_calendar.closes.index.get_loc(start_dt)    
    
dat = data.get_history_window(
    assets=[fut],
    end_dt=end_dt,
    bar_count=end_loc - start_loc,
    frequency='1d',
    field='close',
    data_frequency='daily'
)


/anaconda3/envs/env_zipline/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: offset is deprecated. Use freq instead
  """Entry point for launching an IPython kernel.
/anaconda3/envs/env_zipline/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: offset is deprecated. Use freq instead
  


In [48]:
dat

,Future(0 [A6F18])
2018-01-03 00:00:00+00:00,0.784
2018-01-04 00:00:00+00:00,0.786
2018-01-05 00:00:00+00:00,0.787
